<a href="https://colab.research.google.com/github/anshupandey/ms-generativeai-apr2025/blob/main/code15_RAG_chain_and_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Implementation with OpenAI and LangChain

In [17]:
!pip install langchain-community langchainhub langchain-openai langchain-chroma langchain langchain-experimental langgraph --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 53.9 MB/s eta 0:00:00


In [18]:
!pip install pypdf faiss-cpu --quiet

In [12]:
import os
embedding_model_name = "text-embedding-ada-002"
model_name = "gpt-4o"

In [4]:
doc_paths = ["https://www.morningstar.com/content/dam/marketing/shared/research/methodology/771945_Morningstar_Rating_for_Funds_Methodology.pdf",
             "https://www.morningstar.in/docs/methodology/CategoryDefinitionsIndiaV3.pdf",
             "https://s21.q4cdn.com/198919461/files/doc_downloads/press_kits/2016/Morningstar-Sustainable-Investing-Handbook.pdf"]

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loaders = [PyPDFLoader(pdf, extract_images=False) for pdf in doc_paths]

docs = []

for loader in loaders:
    doc = loader.load()
    docs.extend(doc)

In [6]:
len(docs)

46

In [7]:
# drop pages which have less than 100 characters (e.g. header pages, empty separater pages)
docs = [doc for doc in docs if len(doc.page_content.strip())>100]
len(docs)

45

In [8]:
# FOr all remaining document, checking the average characters count
sum(len(doc.page_content) for doc in docs)/len(docs)

2709.4222222222224

In [9]:
# split the documents(each pdf page as one document) , into multiple so that at max there are 3500 characters in one document, with overlap of 500 characters while splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3500, chunk_overlap=500)
splits = text_splitter.split_documents(docs)
len(splits)

53

In [10]:
print(splits[1].page_content)

3
3
3
©2021 Morningstar, Inc. All rights reserved. The information in this document is the property of Morningstar, Inc. Reproduction or transcription by any means, in whole or in part, without the prior written 
consent of Morningstar, Inc., is prohibited.
 
The Morningstar RatingTM for Funds    August 2021Page 2 of 21
captured by standard deviation, as would be the case if excess return were normally or lognormally 
distributed, which is not always the case. Also, standard deviation measures variation both above 
and below the mean equally. But investors are generally risk-averse and dislike downside variation 
more than upside variation. Morningstar gives more weight to downside variation when calculating 
Morningstar Risk-Adjusted Return and does not make any assumptions about the distribution of 
excess returns.
The other commonly accepted meaning of “risk-adjusted” is based on assumed investor preferences. 
Under this approach, higher return is “good” and higher risk is “bad” und

In [13]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [14]:
# initializng the vector store using CHromaDB
#from langchain_community.vectorstores import FAISS
#from langchain.embeddings import HuggingFaceBgeEmbeddings
#embedding_model_name = "BAAI/bge-large-en-v1.5"
#embeddings = HuggingFaceBgeEmbeddings(model_name=embedding_model_name,)

In [15]:
from langchain_community.vectorstores import FAISS
# Using embedding model, to embed documents to vector and store to a vector db (Inmemory vectorDB - FAISS)
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# using vector db object to initialize a retriever object - to perform vector search/retrieval
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [19]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# using vector db object to initialize a retriever object - to perform vector search/retrieval
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [20]:
retrieved_docs = retriever.invoke("What is Large Cap equity fund")
len(retrieved_docs)

2

In [21]:
print(retrieved_docs[0].page_content)

? 
 
 
 
 
 
 
 
 
 
 
 
Categor y Definitions  
India 
Equity  
 
Large-Cap 
Large-Cap funds primarily consist of stocks which are the Top 100 stocks by full market capitalization  of 
the equity market. These funds invest at least 80% of total assets in Indian equities and the balance can 
be invested in other asset classes such as fixed income and overseas equities, among others. Funds in 
this category would invest at least 80% of their total assets in large-cap stocks. 
Morningstar Category Index: S&P BSE 100 TR 
 
Mid-Cap 
Mid-Cap funds primarily consist of stocks ranked 101st to 250th by full market capitalization of the 
equity market. These funds invest at least 65% of total assets in Indian equities, and the balance can be 
invested in other asset classes such as fixed income and overseas equities, among others. Funds in this 
category would invest at least 65% of their total assets in mid-cap stocks. 
Morningstar Category Index: S&P BSE Mid Cap TR 
 
Small-Cap 
Small-Cap fun

In [22]:
print(retrieved_docs[1].page_content)

©2019 Morningstar, Inc. All rights reserved. The information in this document is the property of Morningstar, Inc. Reproduction or transcription by any means, in whole or part, without  
the prior written consent of Morningstar, Inc., is prohibited. 
Category Definitions , India  | 26 February 2021  Page 2 of 12  
Multi- Cap 
Multi-Cap funds invest at least 75% of their total assets in Indian equities, and the balance can be 
invested in other asset classes such as fixed income and overseas equities, among others. These funds 
will invest a minimum of 25% each in Large Cap, Mid Cap and Small Cap stocks. 
Morningstar Category Index: S&P BSE 500 TR 
 
 
Large & Mid- Cap 
Large & Mid-Cap funds primarily consist of stocks which are the Top 250 stocks by full market 
capitalization of the equity market. These funds invest at least 70% of total assets in Indian equities and 
the balance can be invested in other asset classes such as fixed income and overseas equities, among 
others. Funds in

### Implementing RAG Chain

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

In [24]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm


In [25]:
response = rag_chain.invoke("tell me about mid cap market")

print(response.content)

Mid-Cap funds consist of stocks ranked 101st to 250th by full market capitalization in the equity market. These funds are required to invest at least 65% of their total assets in Indian equities, with the remaining assets potentially allocated to other asset classes such as fixed income and overseas equities. The Morningstar Category Index for Mid-Cap funds is the S&P BSE Mid Cap Total Return Index.


### Implementing RAG Agent

### Creating retriever Tool

In [26]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "searchCapitalMarket",
    "Searches and returns excerpts about trading stocks markets shares capital markets, DO not use it for any other info than that of capital market/ finance questions",
)
tools = [tool,]

In [27]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")


In [43]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools,) #messages_modifier=system_message

response = agent_executor.invoke({"messages": [("human", "Hi, I am Anshu")]})

for k in response['messages']:print(k)


content='Hi, I am Anshu' additional_kwargs={} response_metadata={} id='27cfd54a-a664-4b74-b500-b3b716aa6b25'
content='Hello Anshu! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 82, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BIXK8VsZLl9PC2FGRL3OiPHRMLZ9R', 'finish_reason': 'stop', 'logprobs': None} id='run-59afbf0e-40fb-41e3-b4ea-c831695f8c5a-0' usage_metadata={'input_tokens': 82, 'output_tokens': 13, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [44]:
response = agent_executor.invoke({"messages": [("human", "What is large cap market?")]})

for k in response['messages']:print(k)

content='What is large cap market?' additional_kwargs={} response_metadata={} id='4ec0dcd1-257c-4a9f-86b0-8dac010f25a9'
content='' additional_kwargs={'tool_calls': [{'id': 'call_RZu5VlutWLFfLitbQ1qbRLm3', 'function': {'arguments': '{"query":"large cap market"}', 'name': 'searchCapitalMarket'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 82, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BIXK9Wvy54AnpgedHCr4tkncDkpfr', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-f9c71a78-ee1a-484a-bb85-d3a2c1054531-0' tool_calls=[{'name': 'searchCapitalMarket', 'args': {'query': 'large cap market'}, 'id': 'call_RZu5VlutWLFfLitbQ1qbRLm3', 'type': 'tool_cal

In [46]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="What is large cap market?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_C4cmhzyyRbmeCvAoFNdSSgNy', 'function': {'arguments': '{"query":"large cap market"}', 'name': 'searchCapitalMarket'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 82, 'total_tokens': 100, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6dd05565ef', 'id': 'chatcmpl-BIXL66011LDh3ZQ2PuztfCEE4R9t5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-80a1c8b7-0a6f-4292-9543-113aacbbc3d7-0', tool_calls=[{'name': 'searchCapitalMarket', 'args': {'query': 'large cap market'}, 'id': 'call_C4cmhzyyRbmeCvAoFNdSSgNy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 82, 'output_tokens': 18, 'total_tokens': 100